In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install -U pandas-profiling
!pip install --upgrade pythainlp
!pip install pyLDAvis
!pip install scipy
!pip install pyLDAvis==2.1.2

  Using cached https://files.pythonhosted.org/packages/3b/a3/34519d16e5ebe69bad30c5526deea2c3912634ced7f9b5e6e0bb9dbbd567/pandas_profiling-3.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d8/c2/b407dce6b18d44f52173f6921d62f326e1329dd6d82fa03b380a14612e4c/missingno-0.5.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/84/6a/3b9902f79b81b4f67b6e7497f3d9c9f1e6bd7a7f4e93ccd6bc0aa8f81282/pydantic-1.8.2-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/93/3e/cb354fb2097fcf2fd5b5a342b10ae2a6e9363ba435b64e3e00c414064bc7/tangled_up_in_unicode-0.1.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/68/ad/6c2406ae175f59ec616714e408979b674fe27b9587f79d59a528ddfbcd5b/seaborn-0.11.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/80/21/ae597efc7ed8caaa43fb35062288baaf99a7d43ff0cf66452ddf47604ee6/Jinja2-3.0.1-py3-none-any.whl
  Using cached https://files.pythonhosted.or

ERROR: conda 4.10.1 requires ruamel_yaml_conda>=0.11.14, which is not installed.
ERROR: conda-repo-cli 1.0.4 requires pathlib, which is not installed.
ERROR: glue-core 0.15.6 has requirement mpl-scatter-density>=0.5, but you'll have mpl-scatter-density 0.4 which is incompatible.
ERROR: phik 0.11.2 has requirement joblib>=0.14.1, but you'll have joblib 0.13.2 which is incompatible.
ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\users\\srisudaw\\appdata\\local\\continuum\\anaconda3\\lib\\site-packages\\matplotlib\\backends\\_backend_agg.cp37-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



Requirement already up-to-date: pythainlp in c:\users\srisudaw\appdata\local\continuum\anaconda3\lib\site-packages (2.3.1)


In [3]:
import pythainlp
import pyLDAvis
import gensim
# import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
#import warnings
#warnings.filterwarnings("ignore", category=DeprecationWarning)

C:\Users\srisudaw\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
C:\Users\srisudaw\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [4]:
df = pd.read_csv('CustomerReviews.csv')
print(df.head())

   Review ID                              Restaurant_ID  \
0          1  352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ   
1          2  352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ   
2          3  352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ   
3          4  352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ   
4          5  352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ   

                                          Restaurant  \
0  Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ   
1  Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ   
2  Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ   
3  Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ   
4  Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ   

                               User  \
0  7b16469831074f7abc7824745ee75212   
1                  pakkaramonpondej   
2                           saanowy   
3                           ployynp   
4  665a902b335b434ab489f00f2fbb477e   

                                            Headline  \
0  ที่สำคัญของร้า

In [5]:
df['Restaurant'].unique()

array(['Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ',
       'Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G', 'ข้าน้อยขอชาบู'],
      dtype=object)

In [6]:
df = df[df['Restaurant']=='Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G']

In [7]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = ['😆','🤣',' ', '  ', '\n','\n\n', 'ร้าน', '(', ')' , '           ','–', '!', '!!','-','/','+','"','','%','\u200b','::', ':','+']
screening_words = stopwords + removed_words

def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    if word not in screening_words:
      merged = merged + ',' + word
  return merged[1:]

In [8]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))
df['Headline_tokenized'] = df['Headline'].apply(lambda x: tokenize_with_space(x))
df.tail()

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating,Review_tokenized,Headline_tokenized
11,12,4621tm-shabushi-เดอะมอลล์บางกะปิ-ชั้น-g,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,c08b2776067a4026b882abfc6fca8330,เปลี่ยนจากกุ้งขาดสารอาหารเป็นกุ้งปอกเปลือกแล้ว...,ห่างหายไปนานสำหรับชาบูชิ ตั้งแต่ทางร้านได้ปรับ...,3.0,"ห่างหาย,สำหรับ,ชาบู,ชิ,ขึ้นราคา,แถม,ลด,เวลา,คุ...","กุ้ง,ขาดสารอาหาร,กุ้ง,ปอกเปลือก,..,รู้"
12,13,4621tm-shabushi-เดอะมอลล์บางกะปิ-ชั้น-g,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,a96ec47b6cff4bd1834c90d603ef0a7e,อิ่ม อร่อย ฝุดฝุด,เมื่อหลายวันก่อนนัดหาข้าวทานกับเดอะแกงค์ และก็...,4.0,"วันก่อน,นัด,หา,ข้าว,ทาน,เดอะ,แกงค์,ลงเอย,อาหาร...","อิ่ม,อร่อย,ฝุดฝุด"
13,14,4621tm-shabushi-เดอะมอลล์บางกะปิ-ชั้น-g,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,a6991f1e0e9b4916ab3ca91d86f8e481,จัดให้เต็มคราบกับชั่วโมงครึ่ง@Shabushi,บอกตรงๆว่าหลายครั้งที่เลือกจะกินบุฟเฟต์จะต้องอ...,3.0,"หลายครั้ง,เลือก,กิน,บุฟเฟต์,อาราม,ณ์,รอ,คิว,สุ...","เต็มคราบ,ชั่วโมง,ครึ่ง,@,Shabushi"
14,15,4621tm-shabushi-เดอะมอลล์บางกะปิ-ชั้น-g,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,7eb0c7d8683e407f83cc4d18e51d800d,ชาบู ชาบูชิ,สวัสดีครับวันนี้จะขอมารีวิวร้านชาบูชิ บุฟเฟขวั...,4.0,"สวัสดี,รีวิว,ชาบู,ชิ,บุ,ฟเฟ,ขวัญใจ,คน,เมนู,เลื...","ชาบู,ชาบู,ชิ"
15,16,4621tm-shabushi-เดอะมอลล์บางกะปิ-ชั้น-g,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,qvanqvan,บุฟเฟต์น้ำเดือด สายพานร่อน,ชาบูชิ สาขาเดอะมอลล์บางกะปิ ตั้งอยู่ประตูหน้าห...,4.0,"ชาบู,ชิ,สาขา,เดอะ,มอลล์,กะปิ,ตั้งอยู่,ประตู,หน...","บุฟเฟต์,น้ำ,เดือด,สายพาน,ร่อน"


In [9]:
documents = df['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [10]:
print(dictionary.token2id.keys())

dict_keys(['..', '399', 'กุ้ง', 'ของกิน', 'ขา', 'คุ้ม', 'ชิ', 'ซูชิ', 'ซ่า', 'ตอน', 'ตอนนี้', 'ทาน', 'นิดนึง', 'บาท', 'บู', 'ปุระ', 'ปูอัด', 'ม', 'ราคา', 'ละลานตา', 'สายพาน', 'หิว', 'อาหาร', 'ฮ่า', 'าน', 'เกี๊ยว', 'เท', 'เทียบ', 'เมนู', 'เล่น', 'แพง', 'แล้วก็', 'ไม่งั้น', 'กลาย', 'กี่', 'ชอบ', 'ชั่วคราว', 'ชาบู', 'ชื่นใจ', 'ซะงั้น', 'ญี่', 'ดึก', 'ตบท้าย', 'ตัว', 'ทอด', 'ทานอาหาร', 'นพ', 'นึก', 'น้ะ', 'ปลา', 'ปุ', 'ผิดหวัง', 'ลืม', 'วก', 'สมใจ', 'อย่า', 'อาหารเสริม', 'เติม', 'เป็นมา', 'เป็นหลัก', 'เย็น', 'ไข่', 'ไอติม', '1.15', '4', 'net', 'กลับมา', 'ของคาว', 'ชม.', 'ซุป', 'ดี', 'ตรงนั้น', 'น้ำ', 'บุฟเฟ่ต์', 'ยืน', 'รส', 'ลายตา', 'สด', 'สไลด์', 'อัพเดท', 'อิ่ม', 'เนื้อ', 'เบ', 'เพลิน', 'เลือก', 'ไลน์', '/คน', '1', '15', '359', '5', 'G', 'Net', 'Shabushi', 'กด', 'กรอ', 'กะปิ', 'กิน', 'ของหวาน', 'ขาดสารอาหาร', 'ขึ้นราคา', 'ข่าว', 'คน', 'คนเดียว', 'ครึ่ง', 'คิว', 'คุณภาพ', 'งาน', 'จาน', 'จืด', 'จุ', 'ชั่วโมง', 'ชั้น', 'ชื่น', 'ดู', 'ตลับ', 'ตัก', 'ตับ', 'ตั้งอยู่', 'ติง', 'ติด', 'ถนนใหญ่'

In [11]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]

In [12]:
print(gensim_corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 3), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1)], [(2, 4), (5, 1), (6, 1), (7, 1), (11, 3), (15, 4), (17, 4), (26, 4), (28, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 2), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 2), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1)], [(1, 1), (2, 1), (5, 1), (6, 1), (7, 1), (10, 1), (11, 3), (13, 1), (15, 1), (17, 1), (18, 1), (26, 1), (37, 1), (59, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1)], [(2, 7), (5, 1), (6, 1), (7, 1)

In [13]:
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

In [14]:
print(word_frequencies)

[[('..', 1), ('399', 1), ('กุ้ง', 1), ('ของกิน', 1), ('ขา', 1), ('คุ้ม', 2), ('ชิ', 1), ('ซูชิ', 1), ('ซ่า', 1), ('ตอน', 1), ('ตอนนี้', 1), ('ทาน', 1), ('นิดนึง', 1), ('บาท', 1), ('บู', 1), ('ปุระ', 1), ('ปูอัด', 1), ('ม', 1), ('ราคา', 1), ('ละลานตา', 1), ('สายพาน', 1), ('หิว', 1), ('อาหาร', 3), ('ฮ่า', 1), ('าน', 1), ('เกี๊ยว', 1), ('เท', 1), ('เทียบ', 1), ('เมนู', 1), ('เล่น', 1), ('แพง', 1), ('แล้วก็', 1), ('ไม่งั้น', 1)], [('กุ้ง', 4), ('คุ้ม', 1), ('ชิ', 1), ('ซูชิ', 1), ('ทาน', 3), ('ปุระ', 4), ('ม', 4), ('เท', 4), ('เมนู', 1), ('กลาย', 1), ('กี่', 1), ('ชอบ', 1), ('ชั่วคราว', 1), ('ชาบู', 2), ('ชื่นใจ', 1), ('ซะงั้น', 1), ('ญี่', 1), ('ดึก', 1), ('ตบท้าย', 1), ('ตัว', 1), ('ทอด', 2), ('ทานอาหาร', 1), ('นพ', 1), ('นึก', 1), ('น้ะ', 1), ('ปลา', 1), ('ปุ', 1), ('ผิดหวัง', 1), ('ลืม', 1), ('วก', 1), ('สมใจ', 1), ('อย่า', 1), ('อาหารเสริม', 1), ('เติม', 1), ('เป็นมา', 1), ('เป็นหลัก', 1), ('เย็น', 1), ('ไข่', 1), ('ไอติม', 1)], [('399', 1), ('กุ้ง', 1), ('คุ้ม', 1), ('ชิ', 1), ('ซูชิ

In [15]:
num_topics = 20
chunksize = 1000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

Wall time: 362 ms


In [16]:
print(id2word)

{0: '..', 1: '399', 2: 'กุ้ง', 3: 'ของกิน', 4: 'ขา', 5: 'คุ้ม', 6: 'ชิ', 7: 'ซูชิ', 8: 'ซ่า', 9: 'ตอน', 10: 'ตอนนี้', 11: 'ทาน', 12: 'นิดนึง', 13: 'บาท', 14: 'บู', 15: 'ปุระ', 16: 'ปูอัด', 17: 'ม', 18: 'ราคา', 19: 'ละลานตา', 20: 'สายพาน', 21: 'หิว', 22: 'อาหาร', 23: 'ฮ่า', 24: 'าน', 25: 'เกี๊ยว', 26: 'เท', 27: 'เทียบ', 28: 'เมนู', 29: 'เล่น', 30: 'แพง', 31: 'แล้วก็', 32: 'ไม่งั้น', 33: 'กลาย', 34: 'กี่', 35: 'ชอบ', 36: 'ชั่วคราว', 37: 'ชาบู', 38: 'ชื่นใจ', 39: 'ซะงั้น', 40: 'ญี่', 41: 'ดึก', 42: 'ตบท้าย', 43: 'ตัว', 44: 'ทอด', 45: 'ทานอาหาร', 46: 'นพ', 47: 'นึก', 48: 'น้ะ', 49: 'ปลา', 50: 'ปุ', 51: 'ผิดหวัง', 52: 'ลืม', 53: 'วก', 54: 'สมใจ', 55: 'อย่า', 56: 'อาหารเสริม', 57: 'เติม', 58: 'เป็นมา', 59: 'เป็นหลัก', 60: 'เย็น', 61: 'ไข่', 62: 'ไอติม', 63: '1.15', 64: '4', 65: 'net', 66: 'กลับมา', 67: 'ของคาว', 68: 'ชม.', 69: 'ซุป', 70: 'ดี', 71: 'ตรงนั้น', 72: 'น้ำ', 73: 'บุฟเฟ่ต์', 74: 'ยืน', 75: 'รส', 76: 'ลายตา', 77: 'สด', 78: 'สไลด์', 79: 'อัพเดท', 80: 'อิ่ม', 81: 'เนื้อ', 82: 'เบ', 83

In [17]:
model.show_topic(18)

[('สด', 0.0032224578),
 ('กิน', 0.0032167414),
 ('กุ้ง', 0.0032163137),
 ('ดี', 0.0032135134),
 ('เวลา', 0.0032127602),
 ('คน', 0.0032118028),
 ('สำหรับ', 0.0032112317),
 ('ใช้ได้', 0.0032112133),
 ('นุ่ม', 0.0032099825),
 ('สาขา', 0.0032097115)]

In [18]:
df['topics'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [19]:
# print(df[['Restaurant', 'Review', 'topics', 'score']])
print(df[[ 'Review', 'topics']])

                                               Review  topics
8   มา านที่ขาบูชิต้องมาตอนหิว ไม่งั้นจะไม่คุ้มนะค...      10
9   ใครชอบกุ้งทอดเทมปุระ แค่กุ้งเทมปุระก็คุ้มแล้ว ...       3
10  กลับมาอัพเดทราคาชาบูชิ ตอนนี้อยู่ที่ 399 บาท n...      16
11  ห่างหายไปนานสำหรับชาบูชิ ตั้งแต่ทางร้านได้ปรับ...       5
12  เมื่อหลายวันก่อนนัดหาข้าวทานกับเดอะแกงค์ และก็...      12
13  บอกตรงๆว่าหลายครั้งที่เลือกจะกินบุฟเฟต์จะต้องอ...      14
14  สวัสดีครับวันนี้จะขอมารีวิวร้านชาบูชิ บุฟเฟขวั...      10
15  ชาบูชิ สาขาเดอะมอลล์บางกะปิ ตั้งอยู่ประตูหน้าห...      13


In [20]:
def topic2Array(x):
  topics = ""
  for i in range(len(x)):
    topics += x[i][0]
    #print(topics)
    if i >= 9:
      break;
  return topics

In [21]:
array_topics = []
for i in range(model.num_topics):
  topic_list = topic2Array(model.show_topic(i))
  array_topics.append(topic_list)

print(len(array_topics))
print (array_topics)

20
['กินสดกุ้งดีสำหรับเวลานุ่มเนื้อคนตัว', 'นิตวันก่อนวลงเอยรี่มุมะนาวหนัด', 'หน้าเลือกค่าหม้อชิสายพาน3ปรับปรุงอาหารข้าวปั้น', 'มกุ้งเทปุระทานทอดชาบูตัวคุ้มชิ', 'นิตวันก่อนวลงเอยรี่มุมะนาวหนัด', 'กินสดกุ้งดีเวลาคนสำหรับเนื้อสาขากลับมา', 'อร่อยเลือกนํ้าดูจิ้มต้องการฟเฟลูกค้าบริการทาน', 'หน้าเลือกหม้อค่าทาน3สายพานชิอาหารห้าง', 'นิตวันก่อนวลงเอยรี่มุมะนาวหนัด', 'สดกินกุ้งดีเวลาตัวกลับมาสำหรับสาขาคน', 'คุ้มทานอาหารเมนูอร่อยสายพานชิเลือกนํ้าจาน', 'นิตวันก่อนวลงเอยรี่มุมะนาวหนัด', 'รสชาติทานนุ่มเดอะเบอร์อยุ่บาทวันก่อนสไลด์แล', 'หม้อเลือกหน้าอาหารชิห้างค่า3ทานสายพาน', 'กินรอคนดูผักบุฟเฟต์คิวสดดีสาขา', 'รอบุฟเฟต์คนกินคิวดูผักมอลล์พี่สาม', 'ทานกุ้งปุระเทมชาบูซูชิชิเลือกดี', 'นิตวันก่อนวลงเอยรี่มุมะนาวหนัด', 'สดกินกุ้งดีเวลาคนสำหรับใช้ได้นุ่มสาขา', 'นิตวันก่อนวลงเอยรี่มุมะนาวหนัด']


In [22]:
for i in array_topics:
  print(i)

กินสดกุ้งดีสำหรับเวลานุ่มเนื้อคนตัว
นิตวันก่อนวลงเอยรี่มุมะนาวหนัด
หน้าเลือกค่าหม้อชิสายพาน3ปรับปรุงอาหารข้าวปั้น
มกุ้งเทปุระทานทอดชาบูตัวคุ้มชิ
นิตวันก่อนวลงเอยรี่มุมะนาวหนัด
กินสดกุ้งดีเวลาคนสำหรับเนื้อสาขากลับมา
อร่อยเลือกนํ้าดูจิ้มต้องการฟเฟลูกค้าบริการทาน
หน้าเลือกหม้อค่าทาน3สายพานชิอาหารห้าง
นิตวันก่อนวลงเอยรี่มุมะนาวหนัด
สดกินกุ้งดีเวลาตัวกลับมาสำหรับสาขาคน
คุ้มทานอาหารเมนูอร่อยสายพานชิเลือกนํ้าจาน
นิตวันก่อนวลงเอยรี่มุมะนาวหนัด
รสชาติทานนุ่มเดอะเบอร์อยุ่บาทวันก่อนสไลด์แล
หม้อเลือกหน้าอาหารชิห้างค่า3ทานสายพาน
กินรอคนดูผักบุฟเฟต์คิวสดดีสาขา
รอบุฟเฟต์คนกินคิวดูผักมอลล์พี่สาม
ทานกุ้งปุระเทมชาบูซูชิชิเลือกดี
นิตวันก่อนวลงเอยรี่มุมะนาวหนัด
สดกินกุ้งดีเวลาคนสำหรับใช้ได้นุ่มสาขา
นิตวันก่อนวลงเอยรี่มุมะนาวหนัด
